In [10]:
#importing packages

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
import seaborn as sns
import time
import datetime
import scipy
from sklearn import linear_model
from sklearn.metrics import mean_squared_error,r2_score

import glob
import os

pd.set_option("display.max_columns", None)

In [30]:
# importing the data files
from os.path import join

grrcsvdir1=''
temp_path1=os.path.join(grrcsvdir1,'cse6242_us_stock_2*db.csv')

csvfiles = glob.glob(temp_path1)
dataframes=[]
file_name=[]

 
for csvfile in csvfiles:
    fl_name = csvfile.split('_')[-1].split('db.')[0]+'_clean'
    file_name.append(fl_name)
    dataframes.append(pd.read_csv(csvfile))

display(dataframes[0].head())

,Unnamed: 0,Unnamed: 1,cur_mkt_cap,cntry_of_risk,security_des,name,id_isin,id_cusip,primary_exchange_name,gics_sector_name,gics_industry_name,gics_sub_industry_name,px_last,rsi,macd,boll,px_volume,eqy_sh_out,pe_ratio,dividend_yield,px_to_book_ratio,px_to_sales_ratio,ev_to_ebitda,curr_entp_val,eps_growth,basic_eps_5yr_avg_gr,growth_in_cap,bvps_growth,rd_expend_yr_growth,retention_ratio,net_rev_growth,empl_growth,asset_growth,is_div_per_shr,book_val_per_sh,cf_free_cash_flow,free_cash_flow_per_sh_growth,ebitda,ebit,oper_margin,pretax_margin,return_com_eqy,return_on_asset,cb_is_roic,return_on_cap,asset_turnover,px_to_cash_flow,cf_net_inc,dvd_payout_ratio,cash_gen_to_cash_req,cash_dvd_coverage,cfo_to_sales,cur_ratio,quick_ratio,tot_debt_to_tot_asset,tot_debt_to_com_eqy,acct_rcv_turn,invent_turn,gross_margin,ebitda_to_tot_int_expn,sharpe_ratio,beta
0,12/30/2016,ESTE US Equity,3.062533e+08,US,ESTE,Earthstone Energy Inc,US27032D3044,27032D304,New York,Energy,"Oil, Gas & Consumable Fuels",Oil & Gas Exploration & Production,13.74,55.317765,0.195752,14.462044,28504.0,22289176.0,NaN,NaN,1.121425,7.465222,NaN,2.971393e+08,NaN,NaN,-12.145923,-46.481856,NaN,NaN,NaN,NaN,-23.308882,0.00,12.252269,-31685000.0,68.306714,-131969000.0,-156743992.0,-382.078764,-399.741615,-46.822523,-35.924227,NaN,NaN,0.106791,NaN,-138004000.0,NaN,-0.696653,NaN,-31.713144,0.963045,0.843722,4.406614,5.380951,3.277594,NaN,0.029251,NaN,3.703425,1.273177
1,12/30/2016,OSK US Equity,4.811207e+09,US,OSK,Oshkosh Corp,US6882392011,688239201,New York,Industrials,Machinery,Construction Machinery & Heavy Trucks,64.61,38.620200,0.010913,71.951880,574821.0,74465360.0,20.542995,0.012072,2.416568,0.766213,10.826921,5.335507e+09,0.000000,1.714522,-0.570647,5.560671,-30.290737,73.482155,NaN,3.759398,-0.854432,0.78,26.736269,450400000.0,NaN,492800000.0,364000000.0,5.796917,4.889158,11.132833,4.773617,NaN,9.135968,1.385143,8.228075,216400000.0,26.517845,3.153384,3.771046,9.200216,1.767695,0.932436,18.746953,42.813054,6.917704,4.578917,16.814241,NaN,1.849373,1.406216
2,12/30/2016,MTZ US Equity,3.151363e+09,US,MTZ,MasTec Inc,US5763231090,576323109,New York,Industrials,Construction & Engineering,Construction & Engineering,38.25,53.589481,0.840197,40.194075,391513.0,82388560.0,43.737580,NaN,3.073861,0.653778,16.657508,4.163428e+09,-96.774186,-29.519576,-7.065223,-2.624100,NaN,100.000000,NaN,NaN,-0.977888,0.00,12.443634,127919000.0,-68.961482,249943008.0,86080000.0,1.785805,1.330807,0.088788,0.028112,NaN,0.097153,1.492371,13.286980,908000.0,0.000000,2.206407,NaN,4.853540,1.509423,1.300325,31.634489,99.210798,4.304516,42.600395,12.083064,NaN,2.210618,1.191676
3,12/30/2016,HLI US Equity,2.075482e+09,US,HLI,Houlihan Lokey Inc,US4415931009,441593100,New York,Financials,Capital Markets,Investment Banking & Brokerage,31.12,67.207067,0.919129,31.797201,33421.0,12652379.0,22.585929,0.021208,2.989685,2.743786,12.219101,1.972505e+09,13.599999,NaN,10.451474,-9.479720,NaN,55.087231,NaN,NaN,2.751979,0.64,10.409123,96252000.0,-2.447007,161428000.0,153021000.0,20.229367,19.777904,13.112908,8.620198,NaN,NaN,0.760612,16.880421,85754000.0,44.912769,1.909339,2.226538,14.694816,1.139093,0.917716,5.082738,7.378423,14.999157,NaN,NaN,NaN,3.293827,0.754464
4,12/30/2016,UTMD US Equity,2.736346e+08,US,UTMD,Utah Medical Products Inc,US9174881089,917488108,NASDAQ GS,Health Care,Health Care Equipment & Supplies,Health Care Supplies,72.75,70.741922,1.400446,72.130613,2992.0,3761300.0,21.720146,0.014364,3.765037,6.836079,12.522769,2.420276e+08,8.815666,12.203068,6.318669,6.007717,-6.432749,68.163286,NaN,NaN,3.319818,1.07,19.322521,14750000.0,4.540049,19327000.0,16373000.0,40.903867,41.468472,17.900372,15.489872,NaN,NaN,0.491229,18.198224,12621000.0,31.836714,3.619094,3.141028,37.503747,7.580137,6.627352,0.000000,0.000000,7.608439,3.538204,60.320276,NaN,1.285987,0.603151


In [31]:
#cleaning datafile columns

#file.dtypes

for df in dataframes:
    df.drop(['Unnamed: 1','cntry_of_risk','id_isin','id_cusip','primary_exchange_name','gics_industry_name','gics_sub_industry_name','growth_in_cap','empl_growth','is_div_per_shr','pretax_margin',
            'asset_turnover','cash_gen_to_cash_req','cash_dvd_coverage','acct_rcv_turn','invent_turn','ebitda_to_tot_int_expn','cf_net_inc','cb_is_roic','cfo_to_sales'],axis=1,inplace=True)
    
    df.rename(columns={'Unnamed: 0':'Date','cur_mkt_cap':'Mkt Cap','security_des':'Ticker','gics_sector_name':'Sector','px_last':'Price','px_volume':'Volume',
                      'eqy_sh_out':'Shares Outstanding','pe_ratio':'P/E','dividend_yield':'Yield',
                      'px_to_book_ratio':'P/BV','px_to_sales_ratio':'P/S','ev_to_ebitda':'EV/EBITDA',
                      'curr_entp_val':'EV','eps_growth':'Annual EPS growth','basic_eps_5yr_avg_gr':'5 yr EPS growth',
                      'bvps_growth':'BV/Share growth','rd_expend_yr_growth':'R&D Expense Annual Growth',
                      'book_val_per_sh':'BV per share','cf_free_cash_flow':'FCF','free_cash_flow_per_sh_growth':'FCF Annual growth',
                      'return_com_eqy':'ROE','return_on_asset':'ROA','return_on_cap':'ROC','px_to_cash_flow':'P/CF',},inplace=True)
    
    df.drop(['5 yr EPS growth','R&D Expense Annual Growth','retention_ratio','net_rev_growth','FCF Annual growth'],axis=1,inplace=True)
    
    df['Revenue']=(df['Price']/df['P/S'])*df['Shares Outstanding']
    df['Gross Profit'] = df['Revenue']*df['gross_margin']
    df['Operating Profit'] =  df['Revenue']*df['oper_margin']
    df['EPS'] = (df['Price']/df['P/E'])
    df['Net Income']=df['EPS']*df['Shares Outstanding']
    df['Net Margin'] = df['Net Income']/df['Revenue']
    

dataframes[0].columns.tolist()

['Date',
 'Mkt Cap',
 'Ticker',
 'name',
 'Sector',
 'Price',
 'rsi',
 'macd',
 'boll',
 'Volume',
 'Shares Outstanding',
 'P/E',
 'Yield',
 'P/BV',
 'P/S',
 'EV/EBITDA',
 'EV',
 'Annual EPS growth',
 'BV/Share growth',
 'asset_growth',
 'BV per share',
 'FCF',
 'ebitda',
 'ebit',
 'oper_margin',
 'ROE',
 'ROA',
 'ROC',
 'P/CF',
 'dvd_payout_ratio',
 'cur_ratio',
 'quick_ratio',
 'tot_debt_to_tot_asset',
 'tot_debt_to_com_eqy',
 'gross_margin',
 'sharpe_ratio',
 'beta',
 'Revenue',
 'Gross Profit',
 'Operating Profit',
 'EPS',
 'Net Income',
 'Net Margin']

In [35]:
#handling missing data function
def cleandata(df):
    
    #fill #N/As
    values = {
              
              'Sector': '',
              'Price': 0, #set #N/As to 0
              'rsi': 50, #set #N/As to 50, neither overbought nor oversold
              'macd': 0, #set #N/As to 0
              'boll': -1, #set #N/As to -1, nonsensical answer to indicate data isn't available
              'Volume': 0, #set #N/As to 0
              'Shares Outstanding': 0, 
              'P/E': 0,
              'Yield': 0,
              'P/BV': 0,
              'P/S': 0,
              'EV/EBITDA': 0,
              'EV': 0,
              'Annual EPS growth': 0,
              'BV/Share growth': 0,
              'asset_growth': 0,
              'BV per share': 0,
              'FCF': 0,
              'ebitda': 0,
              'ebit': 0,
              'oper_margin': 0,
              'ROE': 0,
              'ROA': 0,
              'ROC': 0,
              'P/CF': 0,
              'dvd_payout_ratio': 0,
              'cur_ratio': 0,
              'quick_ratio': 0,
              'tot_debt_to_tot_asset': 0,
              'tot_debt_to_com_eqy': 0,
              'gross_margin': 0,
              'sharpe_ratio': 1, #set #N/As to 1 return:risk ratio of stock is "balanced", >1 excess returns to risk
              'beta': 1, #set #N/A to 1 assume correlated with market
              'Revenue':0,
              'Gross Profit':0,
              'Operating Profit':0,
              'EPS':0,
              'Net Income':0,
              'Net Margin':0
             } 
    df = df.fillna(value=values)
    
    #drop rows with no ticker symbol, last trade price, volume, or shares outstanding as they're not readily tradeable
   
    df = df[df.Ticker != '']
    df = df[df.Price != 0]
    df = df[df['Shares Outstanding'] != 0]
    df = df[df.Volume != 0]
    df = df[df.Revenue !=0]
    #df =df[df['Gross Profit']!=0]
    
    
    
    return df
    
   

In [33]:
#checking raw_data for number of Nan's

for i in range(len(dataframes)):
    print ('Length of {}:{}'.format(file_name[i],len(dataframes[i])))
print ('\n')    


column_names=dataframes[0].columns.tolist()
for name in column_names:
    for i in range(len(dataframes)):
        print ('Year:{},Column:{},No.of NaN:{}'.format(file_name[i],name,dataframes[i][name].isna().sum()))
    print ('\n')
    
        
    

Length of 2016_clean:38992
Length of 2017_clean:41891
Length of 2018_clean:45119
Length of 2019_clean:41867
Length of 2020_clean:41733


Year:2016_clean,Column:Date,No.of NaN:0
Year:2017_clean,Column:Date,No.of NaN:0
Year:2018_clean,Column:Date,No.of NaN:0
Year:2019_clean,Column:Date,No.of NaN:0
Year:2020_clean,Column:Date,No.of NaN:0


Year:2016_clean,Column:Mkt Cap,No.of NaN:0
Year:2017_clean,Column:Mkt Cap,No.of NaN:0
Year:2018_clean,Column:Mkt Cap,No.of NaN:0
Year:2019_clean,Column:Mkt Cap,No.of NaN:0
Year:2020_clean,Column:Mkt Cap,No.of NaN:0


Year:2016_clean,Column:Ticker,No.of NaN:0
Year:2017_clean,Column:Ticker,No.of NaN:7
Year:2018_clean,Column:Ticker,No.of NaN:0
Year:2019_clean,Column:Ticker,No.of NaN:0
Year:2020_clean,Column:Ticker,No.of NaN:0


Year:2016_clean,Column:name,No.of NaN:0
Year:2017_clean,Column:name,No.of NaN:0
Year:2018_clean,Column:name,No.of NaN:0
Year:2019_clean,Column:name,No.of NaN:0
Year:2020_clean,Column:name,No.of NaN:0


Year:2016_clean,Column:Sector,

In [36]:
for i in range(len(dataframes)):
    df = cleandata(dataframes[i])
    df.to_csv(file_name[i]+'.csv')

